In [ ]:
import numpy as np
import matplotlib.pyplot as plt

$\theta\sim \frac{1}{2}\delta_{-1}+\frac{1}{2}\delta_{1}$


${\rm mmse}^*(\Sigma^{-2})=E\left[\frac{4}{\big(\exp(\frac{2+2\Sigma \tilde{G}}{\Sigma^2})+1\big)^2}\right]$

${\rm mmse}(\Sigma^{-2}, t)={\rm mmse}^*(\Sigma^{-2}+t)$

$\eta(u; \Sigma^2, z_t, t) = \frac{\exp(2z_t+\frac{2u}{\Sigma^2})-1}{\exp(2z_t+\frac{2u}{\Sigma^2})+1}$

$\eta'(u; \Sigma^2, z_t, t) = \frac{\frac{4}{\tau^2}\exp(2z_t+\frac{2u}{\Sigma^2})}{\big(\exp(2z_t+\frac{2u}{\Sigma^2})+1\big)^2}$

In [10]:
def get_MMSE_star(inverse_Sigma_squared):
    Sigma_squared = 1/inverse_Sigma_squared
    num_simu = 1000
    normal_sam = np.random.normal(loc=0.0, scale=1.0, size=num_simu)
    
    s = (2 * normal_sam * np.sqrt(Sigma_squared) + 2) / Sigma_squared
    s_posi = (s > 0)
    s_nega = (s <= 0)
    
    num = np.zeros(num_simu)
    deno = np.zeros(num_simu)
    
    num[s_posi] = 4 * np.exp(-2 * s[s_posi])
    deno[s_posi] = (np.exp(-s[s_posi]) + 1) ** 2

    num[s_nega] = 4
    deno[s_nega] = (np.exp(s[s_nega]) + 1) ** 2
    
    return (np.mean(num/deno))

def get_MMSE(inverse_Sigma_squared, t):
    return get_MMSE_star(inverse_Sigma_squared+t)



def get_tau_t_squared_series(t, Delta, nu, alpha, K):
    tau_squared_series = np.zeros(K)
    tau_squared_series[0] = (Delta + nu) / alpha
    for k in range(1, K):
        tau_squared_series[k] = (Delta + get_MMSE((tau_squared_series[k-1]) ** (-1), t))/ alpha
    return tau_squared_series



def get_eta(u, Sigma_squared, z_t, t):
    s = 2 * z_t + 2 * u / Sigma_squared
    num = np.zeros(len(s))
    deno = np.zeros(len(s))
    
    s_posi = (s > 0)
    s_nega = (s <= 0)
    
    num[s_posi] = 1 - np.exp(-s[s_posi])
    deno[s_posi] = 1 + np.exp(-s[s_posi])
    
    num[s_nega] = np.exp(s[s_nega]) - 1
    deno[s_nega] = np.exp(s[s_nega]) + 1
    
    return num/deno
        


def get_eta_prime(u, Sigma_squared, z_t, t):
    s = 2 * z_t + 2 * u / Sigma_squared
    
    num = np.zeros(len(s))
    deno = np.zeros(len(s))
    
    s_posi = s > 0
    s_nega = s <= 0
    
    num[s_posi] = 4 / Sigma_squared * np.exp(-s[s_posi])
    deno[s_posi] = (np.exp(-s[s_posi]) + 1) ** 2
    
    num[s_nega] = 4 / Sigma_squared * np.exp(s[s_nega])
    deno[s_nega] = (np.exp(s[s_nega]) + 1) ** 2
    
    return num/deno



def run_AMP(y, z_t, t, phi_matrix, Delta, nu, alpha, K):
    N = phi_matrix.shape[1]
    m = np.zeros(N)
    r = y
    tau_squared_series = get_tau_t_squared_series(t, Delta, nu, alpha, K)
    for k in range(K):
        m = get_eta(np.dot(phi_matrix.T, r) + m, tau_squared_series[k], z_t, t)
        b = sum(get_eta_prime(np.dot(phi_matrix.T, r) + m, tau_squared_series[k], z_t, t)) / (N * alpha)
        r = y - np.dot(phi_matrix, m) + r * b
    return m


def run_alg(y, phi_matrix, K, T, delta, Delta, nu, alpha):
    N = phi_matrix.shape[1]
    L = int(T / delta)
    theta_alg_timeline = np.zeros((L, N))
    z_t_tilde = np.zeros(N)
    theta_alg_timeline[0, :] = z_t_tilde
    for l in range(1, L):
        t = (l+1) * delta
        b_motion = np.random.normal(loc=0, scale=1, size=N)
        m = run_AMP(y, z_t_tilde, t, phi_matrix, Delta, nu, alpha, K)
        z_t_tilde = z_t_tilde + m * delta + np.sqrt(delta) * b_motion
        theta_alg_timeline[l, :] = z_t_tilde/t
    return theta_alg_timeline



def plot_fir_sec(theta_alg_timeline, display_interval):
    plt.axis([-2.2, 2.2, -2.2, 2.2])
    plt.scatter(theta_alg_timeline[0, 0], theta_alg_timeline[0, 1], s=30, marker="8", color="#FCBB44")
    plt.scatter(theta_alg_timeline[::display_interval, 0], theta_alg_timeline[::display_interval, 1], s=2, color="#839DD1")
    plt.plot(theta_alg_timeline[::display_interval, 0], theta_alg_timeline[::display_interval, 1], linewidth=1, color="#839DD1")
    plt.scatter(theta_alg_timeline[-1, 0], theta_alg_timeline[-1, 1], s=110, marker="*", color="#FCBB44")



def plot_fir_sec_multiples(theta_alg_timeline11, 
                           theta_alg_timeline12, 
                           theta_alg_timeline13, 
                           theta_alg_timeline14, 
                           theta_alg_timeline21, 
                           theta_alg_timeline22, 
                           theta_alg_timeline23, 
                           theta_alg_timeline24,
                           theta_alg_timeline31, 
                           theta_alg_timeline32, 
                           theta_alg_timeline33, 
                           theta_alg_timeline34,
                           display_interval, 
                           pth):
    plt.subplot(3, 4, 1)
    plot_fir_sec(theta_alg_timeline11, display_interval)
    ax = plt.gca()
    ax.set_aspect(1)
    
    plt.subplot(3, 4, 2)
    plot_fir_sec(theta_alg_timeline12, display_interval)
    ax = plt.gca()
    ax.set_aspect(1)
    
    plt.subplot(3, 4, 3)
    plot_fir_sec(theta_alg_timeline13, display_interval)
    ax = plt.gca()
    ax.set_aspect(1)
    
    plt.subplot(3, 4, 4)
    plot_fir_sec(theta_alg_timeline14, display_interval)
    ax = plt.gca()
    ax.set_aspect(1)
    
    
    
    plt.subplot(3, 4, 5)
    plot_fir_sec(theta_alg_timeline21, display_interval)
    ax = plt.gca()
    ax.set_aspect(1)
    
    plt.subplot(3, 4, 6)
    plot_fir_sec(theta_alg_timeline22, display_interval)
    ax = plt.gca()
    ax.set_aspect(1)
    
    plt.subplot(3, 4, 7)
    plot_fir_sec(theta_alg_timeline23, display_interval)
    ax = plt.gca()
    ax.set_aspect(1)
    
    plt.subplot(3, 4, 8)
    plot_fir_sec(theta_alg_timeline24, display_interval)
    ax = plt.gca()
    ax.set_aspect(1)
    
    
    
    
    
    plt.subplot(3, 4, 9)
    plot_fir_sec(theta_alg_timeline31, display_interval)
    ax = plt.gca()
    ax.set_aspect(1)
    
    plt.subplot(3, 4, 10)
    plot_fir_sec(theta_alg_timeline32, display_interval)
    ax = plt.gca()
    ax.set_aspect(1)
    
    plt.subplot(3, 4, 11)
    plot_fir_sec(theta_alg_timeline33, display_interval)
    ax = plt.gca()
    ax.set_aspect(1)
    
    plt.subplot(3, 4, 12)
    plot_fir_sec(theta_alg_timeline34, display_interval)
    ax = plt.gca()
    ax.set_aspect(1)
    
    
    
    plt.subplots_adjust(wspace=0.5)
    plt.savefig(pth)
    plt.show()

    
def alg_mse(theta, theta_alg):
    mse = np.zeros(len(theta_alg))
    for i in range(len(theta_alg)):
        mse[i] = np.mean((theta - theta_alg[i]) ** 2)
    return mse



def get_y_phi_theta(M, N, Delta, seed):
    np.random.seed(seed)
    alpha = M/N
    theta = np.random.binomial(1, 1/2, N) * 2 - 1
    phi_matrix = np.random.normal(loc=0.0, scale=np.sqrt(1 / M), size=(M, N))
    w = np.random.normal(loc=0.0, scale=1, size=M)
    y = np.dot(phi_matrix, theta) + np.sqrt(Delta/alpha) * w
    return [y, phi_matrix, theta]



In [3]:
alpha = 0.8
K = 20
T = 200
delta = 0.1
nu = 1

L = int(T/delta)

N = 1250
M = int(N*alpha)

In [4]:
Delta_list = [0.01, 1, 10]


theta_alg_timeline_multi = np.zeros((12, L, N))
for i in range(3):
    Delta = Delta_list[i]
    for j in range(4):
        y, phi_matrix, theta = get_y_phi_theta(M, N, Delta, i*4+j)
        theta_alg_timeline = run_alg(y, phi_matrix, K, T, delta, Delta, nu, alpha)
        theta_alg_timeline_multi[i*4+j,:,:] = theta_alg_timeline

In [5]:
np.save("./data_saved/discrete_M_1000_N_1250_K_20_delta_0_1_T_200_Delta_0_01_1_10", theta_alg_timeline_multi)

In [6]:
theta_alg_timeline_multi_load = np.load("./data_saved/discrete_M_1000_N_1250_K_20_delta_0_1_T_200_Delta_0_01_1_10.npy")

In [ ]:
plot_fir_sec_multiples(theta_alg_timeline_multi_load[0,:,:], 
               theta_alg_timeline_multi_load[1,:,:],
               theta_alg_timeline_multi_load[2,:,:],
               theta_alg_timeline_multi_load[3,:,:], 
               theta_alg_timeline_multi_load[4,:,:], 
               theta_alg_timeline_multi_load[5,:,:],
               theta_alg_timeline_multi_load[6,:,:],
               theta_alg_timeline_multi_load[7,:,:],
               theta_alg_timeline_multi_load[8,:,:], 
               theta_alg_timeline_multi_load[9,:,:],
               theta_alg_timeline_multi_load[10,:,:],
               theta_alg_timeline_multi_load[11,:,:],
               20, "./data_saved/discrete_M_1000_N_1250_K_20_delta_0_1_T_200_Delta_0_01_1_10.png")